# Modeling Notebook
This section focuses on training and evaluating machine learning models using LightGBM. It includes performing hyperparameter tuning with RandomizedSearchCV, visualizing feature importance, analyzing the confusion matrix and assessing overall model performance.

### Import libraries

In [ ]:
from src.model_train import ModelTrainer
from src.model_evaluate import ModelEvaluator
import pandas as pd

### Load data

In [ ]:
df_modelo = pd.read_parquet('~/ifood_case/data/processed/df_model.parquet')

In [ ]:
df_modelo.head()

#### Define columns groups

In [ ]:
categorical_cols = ['last_completed_offer_id', 'gender']  
numerical_cols = ['age','credit_card_limit','time_since_registered','total_orders', 'average_transaction_value',
                  'n_viewed','completion_rate']   
target = 'offer_completed'

#### Train model, retrieve best parameters, and make predictions

In [ ]:
trainer = ModelTrainer(df_modelo, categorical_cols, numerical_cols, target)
random_search, X_test, y_test = trainer.train()
best_params = trainer.get_best_params(random_search)
best_score = trainer.get_best_score(random_search)    

print("Best parameters:", best_params)
print("Best score:", best_score)

y_pred = trainer.predict(random_search,X_test)
y_pred_proba = trainer.predict_proba(random_search,X_test)

### Use **ModelEvaluator** class to display model performance

In [ ]:
evaluator = ModelEvaluator(y_pred,y_pred_proba,y_test,random_search,categorical_cols,numerical_cols)
evaluator.evaluate()